# Ergebnisse

Dieses Notebook enthält die Analyse unserer Ergbnisse sowie den Code zur Auswertung des Models anhand der Testdaten. Das Laden der Daten und das Trainieren des Models dauern jeweils ca. 2 Minuten. Nach dem Trainieren wird ein Dokument mit den Modellgewichten gespeicher.

### Ergebnisse Testdaten

Die Auswertung der Testdaten zeigt eine zufriedenstellende Performance mit Blick auf das gesamte Modell. Sowohl der R2-Wert als auch der MAPE (Mean Absolute Percentage Error) liegen etwas unter den Werten der Validierung, sind jedoch nicht wesentlich schlechter. Interessanterweise scheint es bei der Analyse nach Gemeindetypologien eine Verschiebung der Probleme zu geben. In der Kategorie "Land" erzielt das Modell bessere Ergebnisse auf den Testdaten als in der Validierung, während es bei der Kategorie "Kern" genau umgekehrt ist. 

Bei genauerer Betrachtung der einzelnen Gemeinden treten jedoch unvorhersehbare Probleme auf. Die Gemeinden Emme, Kriens, Menznau, Luzern, Pfaffnau, Ruswil und Willisau weisen nicht nur eine sehr schlechte Performance auf, sondern auch eine unerklärliche. Die Vorhersagen des Modells treffen nicht nur nicht mit der Realität überein, sondern zeigen auch keinerlei Konsistenz. Die prognostizierte Bevölkerung springt von Jahr zu Jahr mit grossen Unterschieden. 

### Interpretation der Ergebnisse und Fehleranalyse 

Die drei bevölkerungsreichsten Gemeinden weisen die schlechteste Performance auf. Das Modell scheint hierbei besondere Schwierigkeiten zu haben. Eine mögliche Erklärung dafür könnte darin liegen, dass diese Gemeinden in ihrer Einzigartigkeit hervorstechen. Im Kanton Luzern gibt es eine Vielzahl kleinerer und mittelgrosser Gemeinden, jedoch keine vergleichbare Gemeinde wie Luzern. Daher besteht der Verdacht, dass das Modell zu stark auf das Vorhersagen von kleineren Gemeinden trainiert wurde und Schwierigkeiten hat, gute Ergebnisse bei grösseren Gemeinden zu erzielen. 

Eine potenzielle Lösung könnte darin bestehen, beim Training Gemeinden ähnlicher Grösse aus anderen Kantonen einzubeziehen. Auf diese Weise könnte das Modell von diesen Gemeinden lernen.  

Der Verdacht, dass das Problem der "dummy variable trap" eine Ursache für die Schwierigkeiten sein könnte, hat sich nicht bestätigt. Nachdem dieser Fehler behoben wurde, ergab sich keine Verbesserung der Performance des Modells. 

Um die Performance der "Problem-Gemeinden" zu verbessern, wurden einige Tests durchgeführt, wobei der Fokus auf die Gemeinde Luzern lag, da sie die schlechtesten Ergebnisse erzielt hat. Dabei stellte sich heraus, dass die Entfernung einiger Merkmale in Verbindung mit dem Ausschluss der Jahre 1991 bis 1998 signifikante Verbesserungen in der Performance bewirkte. Insbesondere die Features "Wohnungen Total" und "Anzahl Privathaushalte" erwiesen sich als ausschlaggebend für diese Verbesserungen. Es lässt sich jedoch nicht abschliessend erklären, warum sich diese beiden Features negativ auf die Performance der Testdaten auswirkte. 

Eine weitere Beobachtung ist, dass sich die Performance der starken Ausreisser verbessert, wenn anstelle der absoluten Bevölkerungszahl die Bevölkerungsdichte vorhergesagt wird. In diesem Szenario zeigt lediglich die Gemeinde Kriens weiterhin sehr schlechte Ergebnisse, während die anderen Gemeinden sich dem Durchschnitt annähern. Es scheint möglich zu sein, dass die Bevölkerungsdichte über den gesamten Datensatz hinweg in einem etwas komprimierteren Spektrum liegt, was es dem Modell ermöglicht, besser damit umzugehen. 

Es kann vermutet werden, dass die Verwendung der Bevölkerungsdichte als Vorhersageziel dem Modell eine gewisse Normalisierung ermöglicht und somit die Performance bei den einwohnerstarken Gemeinden verbessert. Das könnte darauf hindeuten, dass die Bevölkerungsdichte als Merkmal eine bessere Einordnung und Vergleichbarkeit zwischen den Gemeinden ermöglicht als die absolute Bevölkerungszahl. 

Es ist wichtig zu betonen, dass diese Änderungen nur zu Analysezwecken ausgeführt wurden und nicht in das endgültige Modell übernommen wurden. 

### Requirements

Dieses Notebook benötigt das Excel 'Alle_Daten'.

#### Python version

Python version used in this notebook: Python 3.10.2

#### Libaries

In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install ipywidgets

### Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import pickle
import ipywidgets as widgets

from sklearn.metrics import mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

In [ ]:
# define the target column
target = "Ständige Wohnbevölkerung Total"

### Funktionen

Datensatz von Excel einlesen

In [ ]:
def read_excel():
    # Function to read in excel
    column_names_to_load = [
        'Siedlungsfläche_in_%', 
        'Landwirtschafts-fläche_in_%',
        'Betriebe_total',
        'Wohnungen - Total',
        'Anzahl_Privathaushalte',
        'Gemeindename',
        'Gemeindetypologien',
        target]

    # Load all data from excel
    path = 'Data/Preparation/Kennzahlen_aller_Gemeinden/Alle_Daten.xlsx'
    data = pd.read_excel(path, header=[0,1], index_col=[0,1])

    # Extract wanted columns
    column_mask = data.columns.isin(column_names_to_load, level=1)
    data = data.loc[:, column_mask]

    # Drop top level column names
    data = data.droplevel(0,axis=1)

    # Reset index
    data = data.reset_index()
    return data

Definiere die Splits

In [ ]:
def train_test_split(data, test_length):
    # Function for train and test split
    start_year = 1991
    amount_of_years = 30

    # Define length of splits
    train_length = amount_of_years - test_length

    # Define years where train and validation split ends
    train_data_end_year = start_year + train_length

    # Apply splits
    train = data[data['Jahr'] <= train_data_end_year]
    test = data[data['Jahr'] > train_data_end_year]
    
    return train, test

Definiere x und y

In [ ]:
def x_y_split(data):
    # Function for x and y split
    y_column_name = target

    x_column_names = [
        'Jahr',
        'Siedlungsfläche_in_%', 
        'Landwirtschafts-fläche_in_%',
        'Betriebe_total',
        'Wohnungen - Total',
        'Anzahl_Privathaushalte',
        'Gemeindename',
        'Gemeindetypologien'
        ]

    # Execute the y split
    y = data[y_column_name]
    y.name = "y"

    x = data[x_column_names]

    assert len(x) == len(y), 'X and Y split need to have the same length'

    return x, y

Dummyvariable und Skalierung

In [ ]:
def set_dummy_variables(x):
    # Function to set dummie variables
    # Set dummie variables for the column Gemeindetypologien and Gemeindename 
    return pd.get_dummies(x, columns=['Gemeindetypologien', 'Gemeindename'])

def scale(x, scaler):
    # Function to scale the data
    # Store the column names that it can be reapplied after scaling
    x_columns = x.columns

    # Scale the data with the StandardScaler
    if not scaler:
        scaler = StandardScaler()
        x = scaler.fit_transform(x)
    else:
        x = scaler.transform(x)
    
    # Create pandas dataframes from ndArrays and reapply columnnames
    x = pd.DataFrame(x, columns=x_columns)

    return x, scaler

Extrahiere Daten für die Visualisierungen

In [ ]:
def get_jahre(x):
    # Function to get all years from the x set
    return list(x['Jahr'].unique())

def get_gemeinde_and_gemeindetypologien(x):
    # Function to get all municipalities and the dummy variable column names
    gemeinden = x.columns.tolist()[11:]

    # All gemeindetypologien dummy variable column names
    gemeinden_typologien = x.columns.tolist()[6:10]

    return gemeinden, gemeinden_typologien

Definiere Metriken

In [ ]:
def evaluate(y, y_pred):
    # Calculate the R2 and MAPE and print it 
    r2 = r2_score(y, y_pred)
    mape = mean_absolute_percentage_error(y, y_pred)
    print('R2: ', r2)
    print('MAPE: ', mape)

Koeffizienten

In [ ]:
def get_coeff(model):
    # get the coefficients from the diffrent models
    type_switch = {
        SVR: lambda x: x.dual_coef_,
    }

    return type_switch[type(model)](model).tolist()

Plotfunktion für einzelne Gemeinden

In [ ]:
def plot_gemeinde(gemeinde, jahre_train, jahre_test, x_list, y_list, y_pred_list):    
    # Concat the train and validation set for the X data set
    x = pd.concat(x_list, axis=0).reset_index()

    # Concat the train and validation set for the Y data set
    y = pd.concat(y_list, axis=0).reset_index()
    
    # Concat the predicted values from the train and validation set
    y_pred = pd.concat(y_pred_list, axis=0).reset_index()

    # Concat the X, Y, and the predicted values
    visualisation_all_gemeinde_data = pd.concat([pd.DataFrame(x), y, y_pred], axis=1)

    # Extract the data for one gemeinde. Get the rows where the gemeinde dummy variable is positive
    visualisation_gemeinde_data = visualisation_all_gemeinde_data[visualisation_all_gemeinde_data[gemeinde] > 0]

    # Extract the expected and the predicted value
    gemeinde_data_y = visualisation_gemeinde_data['y']
    gemeinde_pred = visualisation_gemeinde_data['pred']

    # Plot the data
    plt.figure()

    # Set title on plot
    plt.title(gemeinde)
    
    # plot the expected data as dots
    plt.scatter(jahre_train, gemeinde_data_y[:len(jahre_train)], label='Train')
    plt.scatter(jahre_test, gemeinde_data_y[len(jahre_train):len(jahre_train) + len(jahre_test)], color='lightgreen', label='Test')

    # plot the prediction as line
    plt.plot(jahre_train + jahre_test, gemeinde_pred, color="black", label='Prediction')

    # Show legend
    plt.legend()

    # activate this for fixed y-axis
    # plt.ylim([50, 2500])

    plt.show()

    # Save the image and close the plot that it doesn't display in the jupyter notebook
    #plt.savefig(f'{folder}/Gemeinden/{gemeinde}.png')
    #plt.close()
      

Plotfunktionen zur Evaluierung

In [ ]:
def plot_std(folder, dataframe, gemeinden_typologien):
    # create a figure with subplots
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10,8))
    fig.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9)


    # iterate over the columns and plot each one in a subplot
    for ax, gemeinden_typologie in zip(axes.flat, gemeinden_typologien):
        # extract the column 
        subset = dataframe[dataframe[gemeinden_typologie] > 0][['Delta_Percent']]
        column_data = subset['Delta_Percent']

        # calculate the mean and standard deviation
        mean = column_data.mean()
        std = column_data.std()

        # plot the density plot with a shaded area for the standard deviation and a line for the mean
        ax = sns.kdeplot(column_data, fill=False, ax=ax, color='red', linewidth=1, label='absolut percentage error', bw_adjust=0.3)
        ax.axvline(mean, color='black', linestyle='dotted', linewidth=1, label = 'mean absolute percentage error: ' + str(round(mean, 1)))
        ax.fill_betweenx([0, ax.get_ylim()[1]], mean-std, mean+std, alpha=0.3, color='blue', label="standard deviation: " + str(round(std, 1)) + " %")

        #labels
        ax.set_xlabel('absolut percentage error')
        ax.set_ylabel('density')
        ax.set_title(gemeinden_typologie, fontsize=10)
        ax.legend()
        ax.grid(False)


    # set same scale for all subplots
    for ax in axes.flat:
        ax.set_xlim([-5, 25])
        ax.set_ylim([0, 0.28])
        ax.tick_params(axis='both', labelsize=8)

    # adjust the layout and show the figure
    fig.tight_layout()
    plt.show()

    #Save the image and close the plot that it doesn't display in the jupyter notebook
    #plt.savefig(f'{folder}/standard_deviation.png')
    #plt.close()

In [ ]:
def plot_density(folder, visualisation_all_gemeindetypologien_data):
    plt.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9)

    # Create density plot
    sns.set_style('whitegrid')
    sns.kdeplot(visualisation_all_gemeindetypologien_data['Delta_Percent'], fill=True, color='red', bw_adjust=0.5)
    
    # Plot mean and median
    mean = np.mean(visualisation_all_gemeindetypologien_data['Delta_Percent'])
    plt.axvline(x=mean, color='black', linestyle='dotted', linewidth=1.3, label='mean absolut percentage error: ' + str(round(mean, 1)))
    median = np.median(visualisation_all_gemeindetypologien_data['Delta_Percent'])
    plt.axvline(x=median, color='blue', linestyle='dotted', linewidth=1.3, label='median absolut percentage error: ' + str(round(median, 1)))
    
    # Calculate 15% and 85% quantiles
    cutoff15 = np.percentile(visualisation_all_gemeindetypologien_data['Delta_Percent'], q=85)
    cutoff85 = np.percentile(visualisation_all_gemeindetypologien_data['Delta_Percent'], q=15)

    # Plot quantiles
    plt.axvline(x=cutoff15, color='red', linestyle='dotted', linewidth=1.3, label='15% quantile: '+ str(round(cutoff85, 1)))
    plt.axvline(x=cutoff85, color='red', linestyle='dotted', linewidth=1.3, label='85% quantile: ' + str(round(cutoff15, 1)))

    # Add labels and title
    plt.xlabel('absolut percentage error', labelpad=10)
    plt.ylabel('density', labelpad=10)
    plt.title('distribution - absolut percentage error', fontsize=10)
    plt.tick_params(axis='both', labelsize=8)
    
    # Show legend
    plt.legend()
    plt.grid(False)

    plt.show()

    # Save the image
    #plt.savefig(f'{folder}/density.png')
    #plt.close()
    

In [ ]:
def plot_violin_plot(folder, gemeinden_typologien, visualisation_all_gemeindetypologien_data):
    plt.figure(figsize=(6, 4))

    # Extract the data for each Gemeindetypologie
    gemeinden_typologien_violin_party = []

    # Loop over all Gemeindetypologien
    for gemeinden_typologie in gemeinden_typologien: 
        gemeinden_typologien_violin_party.append(
            # Extract the data for one Gemeindetypologie. Get the rows where the Gemeindetypologie dummy variable is positive
            visualisation_all_gemeindetypologien_data[visualisation_all_gemeindetypologien_data[gemeinden_typologie] > 0]['Delta_Percent']
        )
    

    # Create violin plot
    violin_parts = plt.violinplot(gemeinden_typologien_violin_party, showmeans=False, showmedians=True)

    # xtick labels
    typologien_list = [''] + [gemeinden_typologie[19:] for gemeinden_typologie in gemeinden_typologien] + ['']

    # Set title
    plt.title("Absolute Percentage error of predictions", fontsize=12, pad=20)

    # Set lables
    plt.xlabel('Gemeindetypologien', fontsize=12, labelpad=10)
    plt.ylabel('Absolute Percentage Error', fontsize=12, labelpad=10)
    plt.xticks(list(range(len(typologien_list))), typologien_list, fontsize=10)
    plt.yticks(fontsize=10)
    plt.grid(True)

    # Set color
    for pc in violin_parts['bodies']:
        pc.set_facecolor('red')
        pc.set_color('red')
        pc.set_edgecolor('black')

    for partname in ('cbars','cmins','cmaxes','cmedians'):
        vp = violin_parts[partname]
        vp.set_edgecolor('red')
        vp.set_linewidth(1)
    
    plt.show()
    
    # Save the image
    #plt.savefig(f'{folder}/topologien.png')
    #plt.close()
    


Funktion zur Datenvorbereitung und Ansteuerung der Plotfunktionen

In [ ]:
def plot_gemeinde_typologie(gemeinden_typologien, x, y, y_pred, folder = None):
    # Concat the X, Y, and the predicted values
    visualisation_all_gemeindetypologien_data = pd.concat([x.reset_index(), y.reset_index(), y_pred.reset_index()], axis=1)
    

    # Calculate delta for each prediction
    visualisation_all_gemeindetypologien_data['Delta'] = visualisation_all_gemeindetypologien_data['pred'] - visualisation_all_gemeindetypologien_data['y']   

    # Calculate the absolute percentage error for each prediction
    visualisation_all_gemeindetypologien_data['Delta_Percent'] = 100
    visualisation_all_gemeindetypologien_data['Delta_Percent'] = (abs(visualisation_all_gemeindetypologien_data['Delta']) / visualisation_all_gemeindetypologien_data['pred'] ) * 100
    #return visualisation_all_gemeindetypologien_data
    print('Violin plot: distribution absolut percentage error per Gemeindetyplogie')
    plot_violin_plot(folder, gemeinden_typologien, visualisation_all_gemeindetypologien_data)
    print('Density plot: distribution absolut percentage error')
    plot_density(folder, visualisation_all_gemeindetypologien_data)
    print('Density plots: distribution absolut percentage error and standar deviation per Gemeindetypologie')
    plot_std(folder, visualisation_all_gemeindetypologien_data, gemeinden_typologien)



### Datenaufbereitung

In [ ]:
TRAIN_LENGTH = 22
TEST_LENGTH = 8 

# Load data if not already loaded
data = read_excel()

# Split
train, test = train_test_split(data, TEST_LENGTH)

x_train, y_train = x_y_split(train)
x_test, y_test = x_y_split(test)

# Set dummy variables
x_train = set_dummy_variables(x_train)
x_test = set_dummy_variables(x_test)

# Extract data for visualisations before scaling
jahre_train = get_jahre(x_train)
jahre_test = get_jahre(x_test)
gemeinden, gemeinden_typologien = get_gemeinde_and_gemeindetypologien(x_test)

# Scale the data
x_train, scaler = scale(x_train, None)
x_test, scaler = scale(x_test, scaler)

### Model trainieren

Die Entscheidung für welches Model und das Tunen der Hyperparamter sind zu diesem Zeitpunkt bereits abgeschlossen.

In [ ]:
# Create model
model_name = f'T{TRAIN_LENGTH} V{TEST_LENGTH} SVR POLY3 C=80 Gamma=0.015'
model = SVR(kernel='poly', degree=3, C=30, gamma=0.1)

print("Trying model:", model_name)


# Train Model
model.fit(x_train, y_train)

# or load already stored weights if desired
# with open(f'model_weights.pkl', 'rb') as f:
#     model = pickle.load(f)

# Save the trained model weights
with open(f'model_weights.pkl', 'wb') as f:
    pickle.dump(model, f)


# Prediction
y_pred_train = pd.Series(model.predict(x_train), name = 'pred')
y_pred_test = pd.Series(model.predict(x_test), name = 'pred')
        

### Model evaluieren

In [ ]:
coeff = get_coeff(model)
# dict comprehension of coeff and x_train.columns
coeff_columns = {k: v for k, v in zip(x_train.columns, *coeff)}

evaluate(y_test, y_pred_test)
print()

print('Coefficients:')
for key in coeff_columns:
    print(key, coeff_columns[key])

### Visualisierung des Outputs

Über das widget können einzelne Gemeinden ausgewählt und betrachtet werden.

In [ ]:
def call_plots(Gemeinde):
    plot_gemeinde(Gemeinde, 
    jahre_train, 
    jahre_test,
    [x_train, x_test], 
    [y_train, y_test], 
    [y_pred_train, y_pred_test]
    )


widgets.interact(call_plots, Gemeinde=widgets.Dropdown(
    options=gemeinden
));

Plots der Evaluierung

In [ ]:
plot_gemeinde_typologie(gemeinden_typologien, 
                        x_test, 
                        y_test,
                        y_pred_test
                            )